# Big data project - Bird images clasification

In [80]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
from sparkdl import readImages as sparkdl_readImages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pyspark
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import os
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat
import tensorflow as tf 

### create a spark session

In [69]:
spark = SparkSession.builder.appName('Bird_Classification').getOrCreate()

### Images load into Python 

In [70]:
bird_df = spark.read.csv(path='birds.csv',
                        sep=',',
                        encoding='UTF-8',
                        comment=None,
                        header=True, 
                        inferSchema=True)
bird_df.show(n=5, truncate=False)

+--------+-----------------------------+---------------+--------+--------------------+
|class id|filepaths                    |labels         |data set|scientific name     |
+--------+-----------------------------+---------------+--------+--------------------+
|0.0     |train/ABBOTTS BABBLER/001.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
|0.0     |train/ABBOTTS BABBLER/007.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
|0.0     |train/ABBOTTS BABBLER/008.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
|0.0     |train/ABBOTTS BABBLER/009.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
|0.0     |train/ABBOTTS BABBLER/002.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
+--------+-----------------------------+---------------+--------+--------------------+
only showing top 5 rows



In [71]:
# tutaj musicie zdefiniować swoje ścieżki, czyli gdzie trzymacie te zdjęcia 
photo_path = "C:/Users/kubam/OneDrive/Desktop/studia/semestr 8/Big data/project/"

In [77]:
bird_df = bird_df.withColumn("filepaths", concat(lit(photo_path), bird_df["filepaths"]))
bird_df.show(truncate=False)

+--------+-----------------------------------------------------------------------------------------------+---------------+--------+--------------------+
|class id|filepaths                                                                                      |labels         |data set|scientific name     |
+--------+-----------------------------------------------------------------------------------------------+---------------+--------+--------------------+
|0.0     |C:/Users/kubam/OneDrive/Desktop/studia/semestr 8/Big data/project/train/ABBOTTS BABBLER/001.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
|0.0     |C:/Users/kubam/OneDrive/Desktop/studia/semestr 8/Big data/project/train/ABBOTTS BABBLER/007.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
|0.0     |C:/Users/kubam/OneDrive/Desktop/studia/semestr 8/Big data/project/train/ABBOTTS BABBLER/008.jpg|ABBOTTS BABBLER|train   |MALACOCINCLA ABBOTTI|
|0.0     |C:/Users/kubam/OneDrive/Desktop/studia/semestr 8/Big data/project/train/

In [93]:
filepaths_train_list = [row["filepaths"] for row in bird_df.filter(bird_df["data set"] == "train").select('filepaths').collect()]
filepaths_test_list = [row["filepaths"] for row in bird_df.filter(bird_df["data set"] == "test").select('filepaths').collect()]
filepaths_valid_list = [row["filepaths"] for row in bird_df.filter(bird_df["data set"] == "valid").select('filepaths').collect()]

print(len(filepaths_train_list)) #correct 
print(len(filepaths_test_list)) #correct 
print(len(filepaths_valid_list)) #correct 

84635
2625
2625
